In [1]:
import src.utils.utilities as utilities
import src.utils.models as models
from api import Api
mlApi = Api()

Using TensorFlow backend.


In [2]:
# define dataset specifics
filename = "../HX-condition-monitoring/datasets/D/data.csv"

columns = [
    ['20TT001', 'Gas side inlet temperature', 'degrees'],
    ['20PT001', 'Gas side inlet pressure', 'barG'],
    ['20FT001', 'Gas side flow', 'M^3/s'],
    ['20TT002', 'Gas side outlet temperature', 'degrees'],
    ['20PDT001', 'Gas side pressure difference', 'bar'],
    ['50TT001', 'Cooling side inlet temperature', 'degrees'],
    ['50PT001', 'Cooling side inlet pressure', 'barG'],
    ['50FT001', 'Cooling side flow', 'M^3/s'],
    ['50TT002', 'Cooling side outlet temperature', 'degrees'],
    ['50PDT001', 'Cooling side pressure differential', 'bar'],
    ['50TV001', 'Cooling side valve opening', '%'],
]

irrelevantColumns = [
    '50FT001',
    '50PDT001',
    '20PDT001',
    '50TV001',
]

targetColumns = [
    '50TT002',
]

In [3]:
traintime = [
        ["2020-01-01 00:00:00", "2020-02-01 00:00:00"],
    ]

testtime = [
    "2020-01-01 00:00:00",
    "2020-07-01 00:00:00"
]

In [4]:
args = utilities.Args({
    'activation': 'relu',
    'loss': 'mean_squared_error',
    'optimizer': 'adam',
    'metrics': ['mean_squared_error'],
    'epochs': 200,
    'batchSize': 32,
    'verbose': 0,
    'callbacks': utilities.getBasicCallbacks(),
    'enrolWindow': 0,
    'validationSize': 0.2,
    'testSize': 0.2,
})

In [5]:
names = list(map(lambda el: el[0], columns))
descriptions = list(map(lambda el: el[1], columns))
units = list(map(lambda el: el[2], columns))

relevantColumns = list(filter(lambda col: col not in irrelevantColumns, map(lambda el: el[0], columns)))
columnUnits = dict(zip(names, units))
columnDescriptions = dict(zip(names, descriptions))

df = mlApi.initDataframe(filename, relevantColumns, columnDescriptions)
df_train, df_test = mlApi.getTestTrainSplit(traintime, testtime)
X_train, y_train, X_test, y_test = mlApi.getFeatureTargetSplit(targetColumns)

Date index set
Columns before removal
-------------------------------------------
Dataset columns:
Col. 0 : Index
Col. 1 : 20TT001 - Gas side inlet temperature
Col. 2 : 20PT001 - Gas side inlet pressure
Col. 3 : 20FT001 - Gas side flow
Col. 4 : 20TT002 - Gas side outlet temperature
Col. 5 : 20PDT001 - Gas side pressure difference
Col. 6 : 50TT001 - Cooling side inlet temperature
Col. 7 : 50PT001 - Cooling side inlet pressure
Col. 8 : 50FT001 - Cooling side flow
Col. 9 : 50TT002 - Cooling side outlet temperature
Col. 10 : 50PDT001 - Cooling side pressure differential
Col. 11 : 50TV001 - Cooling side valve opening
-------------------------------------------
Columns after removal
-------------------------------------------
Dataset columns:
Col. 0 : 20TT001 - Gas side inlet temperature
Col. 1 : 20PT001 - Gas side inlet pressure
Col. 2 : 20FT001 - Gas side flow
Col. 3 : 20TT002 - Gas side outlet temperature
Col. 4 : 50TT001 - Cooling side inlet temperature
Col. 5 : 50PT001 - Cooling side in

In [6]:
mlp_10 = models.kerasSequentialRegressionModel(
    params={
        'name': '10 normal',
        'X_train': X_train,
        'y_train': y_train,
        'args': args,
    },
    structure=[
        [10, args.activation]
    ],
)

mlp_20 = models.kerasSequentialRegressionModel(
    params={
        'name': '20 normal',
        'X_train': X_train,
        'y_train': y_train,
        'args': args,
    },
    structure=[
        [20, args.activation]
    ],
)

mlp_128 = models.kerasSequentialRegressionModel(
    params={
        'name': '128 normal',
        'X_train': X_train,
        'y_train': y_train,
        'args': args,
    },
    structure=[
        [128, args.activation]
    ],
)

mlp_10_10 = models.kerasSequentialRegressionModel(
    params={
        'name': '10 10 normal',
        'X_train': X_train,
        'y_train': y_train,
        'args': args,
    },
    structure=[
        [10, args.activation],
        [10, args.activation]
    ],
)

mlp_32_32 = models.kerasSequentialRegressionModel(
    params={
        'name': '32 32 normal',
        'X_train': X_train,
        'y_train': y_train,
        'args': args,
    },
    structure=[
        [32, args.activation],
        [32, args.activation]
    ],
)

mlp_128_128 = models.kerasSequentialRegressionModel(
    params={
        'name': '128 128 normal',
        'X_train': X_train,
        'y_train': y_train,
        'args': args,
    },
    structure=[
        [128, args.activation],
        [128, args.activation]
    ],
)

sklearnLinearModel = models.sklearnRidgeCV(
    params={
        'name': 'Linear',
        'X_train': X_train,
        'y_train': y_train,
    },
) 

In [7]:
modelList = [
    mlp_10,
    mlp_20,
    mlp_128,
    mlp_10_10,
    mlp_32_32,
    mlp_128_128,
    sklearnLinearModel,
]

mlApi.initModels(modelList)
retrain=False
mlApi.trainModels(retrain)
modelNames, metrics_train, metrics_test = mlApi.predictWithModels(plot=True)

Training model 10 normal


InternalError:  Blas GEMM launch failed : a.shape=(32, 6), b.shape=(6, 10), m=32, n=10, k=6
	 [[node dense_1/MatMul (defined at c:\users\herma\apps\master-thesis\venv\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_1010]

Function call stack:
keras_scratch_graph
